# Short introduction to the AAU IoT API

The API consists of two classes you will have to interact with:  
`aau_iot`   which interacts with sensors and modem for communication over MQTT.  
`MqttData`  is the dataclass, which are serialized and sent over the wire.

In [3]:
import time
from aauiot import aau_iot, MqttData

server_ip = "130.225.37.241"
group_name = "AAU_IoT"

iot = aau_iot(server_ip, group_name)

time_now = iot.get_time()
print(time_now)

16:08:58


## Interaction with sensors

Once the aau_iot object has been instantiated it is possible to interact with any connected sensors

The platform consists of three sensors
- VEML7700 used for measuring light
- BME680 used for temperature, humidity and pressure measurement (The internal gas sensor is disabled)
- SGP30 is used for air quality

Each sensor returns its value and timestamp for measurement

In [60]:
print("**BME680**")
temp, ts = iot.temperature()
print(f"{ts}  Temperature: {temp:.2f} [degC]")
humidity, ts = iot.humidity()
print(f"{ts}  Humidity:    {humidity:.2f} [%]")
pressure, ts = iot.pressure()
print(f"{ts}  Pressure:    {pressure:.2f} [hPa]\n")

print("**SGP30**")
gas, ts = iot.gas()
print(f"{ts}  TVOC: {gas[0]} [ppb], eCO2: {gas[1]} [ppm]\n")

print("**VEML7700**")
light, ts = iot.light()
print(f"{ts}  light: {light:.2f} [Lux]")
light_raw, ts = iot.light.get_light_raw()
print(f"{ts}  raw:   {light_raw:.2f} [Lux]")



**BME680**
15:58:05  Temperature: 25.70 [degC]
15:58:05  Humidity:    52.97 [%]
15:58:05  Pressure:    1011.13 [hPa]

**SGP30**
15:58:05  TVOC: 3 [ppb], eCO2: 400 [ppm]

**VEML7700**
15:58:05  light: 66.36 [Lux]
15:58:05  raw:   65.86 [Lux]


### Sensor Settings

Some sensors allows the user to set gain and oversampling parameters

The BME680 supports setting the oversampling rate  
VEML7700 supports setting gain and integration time  
SGP30 has no additional options.

In [47]:
# Temperature:
iot.temperature.set_oversample_rate('ovr_samp_0')
iot.temperature.set_oversample_rate('ovr_samp_1')
iot.temperature.set_oversample_rate('ovr_samp_2')
iot.temperature.set_oversample_rate('ovr_samp_4')
iot.temperature.set_oversample_rate('ovr_samp_8')
iot.temperature.set_oversample_rate('ovr_samp_16')

# Pressure
iot.pressure.set_oversample_rate('ovr_samp_0')
iot.pressure.set_oversample_rate('ovr_samp_1')
iot.pressure.set_oversample_rate('ovr_samp_2')
iot.pressure.set_oversample_rate('ovr_samp_4')
iot.pressure.set_oversample_rate('ovr_samp_8')
iot.pressure.set_oversample_rate('ovr_samp_16')

iot.humidity.set_oversample_rate('ovr_samp_0')
iot.humidity.set_oversample_rate('ovr_samp_1')
iot.humidity.set_oversample_rate('ovr_samp_2')
iot.humidity.set_oversample_rate('ovr_samp_4')
iot.humidity.set_oversample_rate('ovr_samp_8')
iot.humidity.set_oversample_rate('ovr_samp_16')

In [46]:
print("Set VEML7700 Gain:")
iot.light.gain = iot.light.gain_val.ALS_GAIN_1_8
print(iot.light.gain)
iot.light.gain = iot.light.gain_val.ALS_GAIN_1_4
print(iot.light.gain)
iot.light.gain = iot.light.gain_val.ALS_GAIN_1
print(iot.light.gain)
iot.light.gain = iot.light.gain_val.ALS_GAIN_2
print(iot.light.gain)

print("\nSet VEML7700 integration time:")
iot.light.integration_time = iot.light.int_time_val.ALS_25MS
print(iot.light.integration_time)
iot.light.integration_time = iot.light.int_time_val.ALS_50MS
print(iot.light.integration_time)
iot.light.integration_time = iot.light.int_time_val.ALS_100MS
print(iot.light.integration_time)
iot.light.integration_time = iot.light.int_time_val.ALS_200MS
print(iot.light.integration_time)
iot.light.integration_time = iot.light.int_time_val.ALS_400MS
print(iot.light.integration_time)
iot.light.integration_time = iot.light.int_time_val.ALS_800MS
print(iot.light.integration_time)

Set VEML7700 Gain:
0.125
0.25
1
2

Set VEML7700 integration time:
25
50
100
200
400
800


## Sending messages with MQTT

In [ ]:
# Initialize MQTT
iot.mqtt_connect("IP") # Connect over RPI network
iot.mqtt_connect("NBIoT") # Connect over SIM7020E modem

### Generate data object

`MqttData` objects can be instantiated in multiple ways:

1. Identifier, no data, no timestamp
2. Identifier, list of data, 1 timestamp
3. Identifier, list of data, list of timestamps
    - Where the length of data and timestamps are the same.

For 1. the object is only instantiated where data and timestamp(s) has to be added afterwards.  

2 Transmits a single timestamps, which will be added to all the data points at the MQTT server.  

3 combines each data point with a timestamp 

In [10]:
print("Case 1 - Single timestamp:")
identifier = "Case1"
data_obj = MqttData(identifier)
data_obj.add_measurement(10, "10:10:10")
data_obj.add_measurement(20)
data_obj.add_measurement(30)
print(data_obj.vals)
print(data_obj.ts)


print("\nCase 1 - Multiple timestamps")
data_obj = MqttData(identifier)
data_obj.add_measurement(10, "10:10:10")
data_obj.add_measurement(20, "10:10:20")
print(data_obj.vals)
print(data_obj.ts)

Case 1 - Single timestamp:
[10, 20, 30]
['10:10:10']

Case 1 - Multiple timestamps
[10, 20]
['10:10:10', '10:10:20']


`data_obj.add_measurement` return the reamining space in the object, until the 512B buffer which can be sent over the Modem is full.

In [18]:
print(f"Remaning space: {data_obj.add_measurement(30, '10:10:30')}")

print(f"Size of object: {len(data_obj)}")

Remaning space: 405
Size of object: 107


In [11]:
print("Case 2 - Single timestamp")
data = [1.23, 1, 24, 24.444, 10]
timestamps = "10:10:10"
identifier = "Test"

data_obj = MqttData(identifier, data, timestamps)
print(data_obj.vals)
print(data_obj.ts)

Case 2 - Single timestamp
[1.23, 1, 24, 24.444, 10]
['10:10:10']


In [13]:
print("Case 3 - Multiple timestamp")
data = [1.23, 1, 24, 24.444, 10]
timestamps = ["10:10:10", "10:10:20", "10:10:30", "10:10:40", "10:10:50",]
identifier = "Test"

data_obj = MqttData(identifier, data, timestamps)
print(data_obj.vals)
print(data_obj.ts)

Case 3 - Multiple timestamp
[1.23, 1, 24, 24.444, 10]
['10:10:10', '10:10:20', '10:10:30', '10:10:40', '10:10:50']


## Transmit object



In [ ]:
iot.mqtt.send_topics(data_obj)